<a href="https://colab.research.google.com/github/davidlope/DSF-compendium-GC/blob/main/batchfragmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas rdkit_pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 29.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from io import StringIO
from rdkit import Chem
from collections import Counter
from rdkit.Chem import rdMolDescriptors, Descriptors3D, AllChem
from rdkit.Chem.Descriptors import MolWt
import numpy as np 

A couple of utility functions

In [ ]:
def clear_substructure_match(mol):
    """Clear substructure highlights
    :param mol: input molecule
    :return: None
    """
    mol.__sssAtoms = []
    
def clear_atom_labels(mol):
    """Clear atom labels
    :param mol: input molecule
    :return: None
    """
    label_list = [""]*mol.GetNumAtoms()
    uru.label_atoms(mol, label_list)

Read the input data from an Excel sheet

In [ ]:
df = pd.read_excel("/content/sympaper-boil database.xlsx")

In [ ]:
df

,CAS,Name,smile,dhv at tb,Tb,dhv at tb average,dhv at tb median,dhv at tb SD,Tb average,Tb median,Tb SD,dsboil,compound source,dh source,Temp source,Formula
0,638-68-6,n-triacontane,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,"[80.12, 80.12]","[722.8, 722.8, 722.9]",80.120000,80.120000,0.000000,722.833333,722.800,0.047140,110.846707,"['mohammadi 2007', 'Fang 2003', 'doaa 2018']","['mohammadi 2007', 'Fang 2003']","['mohammadi 2007', 'Fang 2003', 'doaa 2018']",NaN
1,84-74-2,Dibutyl phthalate,CCCCOC(=O)c1ccccc1C(=O)OCCCC,79.171616,613.2,79.171616,79.171616,0.000000,613.200000,613.200,NaN,129.112225,benk 2014,benk 2014,https://pubchem.ncbi.nlm.nih.gov/compound/Dibu...,NaN
2,630-03-5,n-nonacosane,CCCCCCCCCCCCCCCCCCCCCCCCCCCCC,"[78.73, 78.73]","[714.0, 714.0, 714.0]",78.730000,78.730000,0.000000,714.000000,714.000,0.000000,110.266106,"['mohammadi 2007', 'Fang 2003', 'doaa 2018']","['mohammadi 2007', 'Fang 2003']","['mohammadi 2007', 'Fang 2003', 'doaa 2018']",NaN
3,630-02-4,n-octacosane,CCCCCCCCCCCCCCCCCCCCCCCCCCCC,"[77.3, 77.3]","[704.8, 704.8, 704.6]",77.300000,77.300000,0.000000,704.733333,704.800,0.094281,109.676504,"['mohammadi 2007', 'Fang 2003', 'doaa 2018']","['mohammadi 2007', 'Fang 2003']","['mohammadi 2007', 'Fang 2003', 'doaa 2018']",NaN
4,593-49-7,n-heptacosane,CCCCCCCCCCCCCCCCCCCCCCCCCCC,"[75.8, 75.8]","[695.4, 695.4, 695.0]",75.800000,75.800000,0.000000,695.266667,695.400,0.188562,109.002013,"['mohammadi 2007', 'Fang 2003', 'doaa 2018']","['mohammadi 2007', 'Fang 2003']","['mohammadi 2007', 'Fang 2003', 'doaa 2018']",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,75-72-9,Chlorotrifluoromethane,FC(F)(F)Cl,"[15.794337594074783, 15.521834507362001]",191.7,15.658086,15.658086,0.136252,191.700000,191.700,0.000000,81.680157,"['benk 2014', 'abooali 2014', 'doaa 2018']","['benk 2014', 'abooali 2014']",doaa 2018,CClF3
1266,74-84-0,Ethane,CC,"[14.716017, 14.71, 14.706760000000001, 14.69, ...","[184.55, 184.55]",14.703822,14.707982,0.010188,184.550000,184.550,0.000000,79.696462,"['zhao 1999', 'liu 2001', 'benk 2014', 'jia 20...","['zhao 1999', 'liu 2001', 'benk 2014', 'jia 20...","['zhao 1999', 'myrdal 1996']",C2H6
1267,74-85-1,Ethene,C=C,"[13.5399504, 13.54, 13.52515111695138, 13.53, ...","[169.44, 169.35]",13.523002,13.530532,0.023495,169.395000,169.395,0.045000,79.875631,"['zhao 1999', 'liu 2001', 'benk 2014', 'jia 20...","['zhao 1999', 'liu 2001', 'benk 2014', 'jia 20...","['zhao 1999', 'myrdal 1996']",C2H4
1268,75-73-0,Tetrafluoromethane,FC(F)(F)F,"[11.826341571072001, 11.814]",145.1,11.820171,11.820171,0.006171,145.100000,145.100,0.000000,81.462238,"['abooali 2014', 'doaa 2018']","['abooali 2014', 'doaa 2018']",doaa 2018,CF4


In [ ]:
df.smile.head()

0    CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
1      CCCCOC(=O)c1ccccc1C(=O)OCCCC
2     CCCCCCCCCCCCCCCCCCCCCCCCCCCCC
3      CCCCCCCCCCCCCCCCCCCCCCCCCCCC
4       CCCCCCCCCCCCCCCCCCCCCCCCCCC
Name: smile, dtype: object

In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define a set of SMARTS rules for matching atom types

In [ ]:
rules = """

xCH3,[C;$([CH3](A))]
xCH2,[C;$([CH2](A))&!$([CH2]([#6^2])(A))&!$([CH2]([#6^1])(A))&!$([CR1H2]=*)&!$([CR1H2](A)(A))]
xCH,[C;$([CH1](A))&!$([CH1]([#6^2])(A)(A))&!$([CH1]([#6^1])(A)(A))&!$([CH1](A)=A)&!$([CR1H1]=*)]
xC,[C;$([CH0](A))&!$([CR1H0]*)&!$([CH0]([#6^1]))&!$([CH0]([#6^2]))&!$([CH0](=O))&!$([CH0](=N))&!$([CH0](=O)([#6^3])(O([CH0](=O)([#6^3]))))&!$([CR1H0](=O)([#6^3])(O([CH0](=O)([#6^3]))))&!$([CR1H0](=O)([#6^3])(O([CH0](=O)([#6^2]))))]
xF,[F;$([F](A))&!$([F]([#6^2]))&!$([F]([#6^1]))]
yF,[F;$([F][#6^2])]
zF,[F;$([F][#6^1])]
xCl,[Cl;$([Cl](A))&!$([Cl]([#6^2]))&!$([Cl]([#6^1]))]
yCl,[Cl;$([Cl][#6^2])]
zCl,[Cl;$([Cl][#6^1])]
xBr,[Br;$([Br](A))&!$([Br]([#6^2]))&!$([Br]([#6^1]))]
yBr,[Br;$([Br][#6^2])]
zBr,[Br;$([Br][#6^1])]
xI,[I;$([I](A))&!$([I]([#6^2]))&!$([I]([#6^1]))]
yI,[I;$([I][#6^2])]
zI,[I;$([I][#6^1])]
yCH3,[C;$([CH3]([#6^2]))]
yCH2,[C;$([CH2]([#6^2])(A))&!$([CH2]([#6^1])(A))]
yCH,[C;$([CH1]([#6^2])(A)([*^3]))&!$([CH1]([#6^2])([#6^1])(A))&!$([CH1](A)=A)&!$([CR1H1]=*)]
yC,[C;$([CH0]([#6^2])([*^3])([*^3])([*^3]))]
yyCH2,[C;$([CH2]([#6^2])([#6^2]))]
yyCH,[C;$([CH1]([#6^2])([#6^2])([*^3]))]
yyC,"[C;$([CH0]([#6^2])([#6^2])([*^3])([*^3])),$([CH0]([#6^2])([#6^2])([OH1])([*^3])),$([CH0]([#6^2])([#6^2])(O([CH3]))(O([CH3])))]"
yyyC,[C;$([CH0]([#6^2])([#6^2])([#6^2])([*^3]))]
yyyyC,[C;$([CH0]([#6^2])([#6^2])([#6^2])([#6^2]))]
zCH3,[C;$([CH3]([*^1]))]
zCH2,[C;$([CH2]([*^1])(A))&!$([CH2]([*^1])([#6^2]))]
zCH,[C;$([CH1]([*^1])([*^3])([*^3]))]
zC,[C;$([CH0]([*^1])([*^3])([*^3])([*^3]))]
yzC,[C;$([CH0]([*^1])([#6^2])([*^3])([*^3]))]
yzCH,[C;$([CH1]([*^1])([#6^2])([*^3]))]
yzCH2,[C;$([CH2]([*^1])([#6^2]))]
rC,[C;$([CR1H0](A)(A)(A)(A))&!$([CR1H0](*)(*)=O)&!$([CR1H0](*)(*)=A)&!$([CR1H0](a))&!$([CR1H0]([#6^2]))&!$([CR1H0]([#6^1]))]
rrC,[C;$([CR2H0]([#6^3])([#6^3])([#6^3])([#6^3]))]
rrCH,[C;$([CR2H1]([#6^3])([#6^3])([#6^3]))]
rCH,[C;$([CR1H1](A)([#6^3])([#6^3]))]
rCH2,[C;$([CR1H2](A)(A))]
rC=,[C;$([CR1H0](A)([#6^3])=A)&!$([CR1H0](*)(*)=O)]
rCH=,[C;$([CR1H1]([#6^3])=A)]
aC=O,[c;$([cR1H0](=O))]
rO,[O;$([OR1H0])&!$([OR1H0]([CH0](=O)([#6^2]))([CH0](=O)([#6^2])))&!$([OR1H0]([CH0](=O)([#6^3]))([#6^2]))&!$([OR1H0]([CH0](=O)([#6^3]))([CH0](=O)([#6^3])))&!$([OR1H0]([CH0](=O)(O))([#6^3]))]
rS,[S;$([SR1H0])]
CH#,[C;$([CH1]#*)]
xC#,[C;$([CH0](#*)([*^3]))]
yC#,[C;$([CH0](#*)([#6^2]))]
CH2=,[C;$([CH2]=*)]
xCH=,[C;$([CH1](A)=A)&!$([CR1H1]=*)&!$([CH1](=O)(O([#6^3])))]
yCH=,[C;$([CH1]([#6^2])=A)&!$([CR1H1]=*)]
xC=,[C;$([CH0](A)(A)=A)&!$([CR1H0]=*)&!$([CH0]([A^2])([A^3])=A)&!$([CH0](A)(A)=O)&!$([CH0](A)(A)=N)&!$([CH0]([A^2])([A^2])=A)&!$([CH0]([A^1])([A^2])=A)&!$([CH0]([A^1])([A^3])=A)&!$([CH0]([A^1])([A^1])=A)]
yC=,[C;$([CH0]([#6^2])([A])=A)&!$([CR1H0]=*)&!$([CH0]([#6^2])([A^2])=A)&!$([CH0]([#6^2])([A^1])=A)]
=C=,[C;$([CH0](=A)(=A))]
aCH,[c;$([cH1](a)(a))]
aC,[c;$([cH0](*)(a)(a))&!$([cR2H0])&!$([cH0]([#6^2])(a)(a))&!$([cH0]([#6^1])(a)(a))&!$([cR3H0])]
aaC,[c;$([cR2H0](a)(a)(a))]
aaCH,[c;$([cR2H1](a)(a))]
aaaC,[c;$([cR3H0](a)(a)(a))]
aO,[o;$([oH0])]
aN,[n;$([nH0])]
aNH,[n;$([nH1])]
aS,[s;$([SH0])]
yOH,[O;$([OH1]([#6^2]))&!$([OH1]([#6](=O)*))]
xOH,[O;$([OH1]([#6^3]))&!$([OH1]([#6](=O)*))]
xO,[O;$([OH0]([#6^3])([#6^3]))&!$([OH0]([#6](=O)*))]
yO,[O;$([OH0]([#6^2])([#6^3]))&!$([OH0]([#6](=O)*))]
yyO,[O;$([OH0]([#6^2])([#6^2]))&!$([OH0]([#6](=O)))&!$([OR1H0])]
yC=OOH,[C;$([CH0](=O)([#6^2])([OH1]))]
xC=OOH,[C;$([CH0](=O)([#6^3])([OH1]))]
xC=OO,[C;$(C([#6^3])(=O)([OH0][*^3]))&!$([CR1H0](=O)([#6^3])(O([#6^3])))]
yC=OO,[C;$(C([#6^2])(=O)([OH0][*^3]))]
xC=O,[C;$(C([#6^3])([#6^3])(=O))]
yC=O,[C;$(C([#6^2])([*^3])(=O))]
xNO2,[N;$(N([#6^3])(=O)[O-])]
yNO2,[N;$(N([#6^2])(=O)[O-])]
yS,[S;$([SH0]([#6^2])([*^3]))]
ySH,[S;$([SH1]([#6^2]))]
xS,[S;$([SH0]([#6^3])([*^3]))]
xSH,[S;$([SH1]([*^3]))]
xNH2,[N;$([NH2][*^3])&!$([NH2]([#16](=O)))&!$([NH2]([#6](=O)))]
xNH,[N;$([NH1]([*^3])([*^3]))&!$([NH1]([#16](=O)))&!$([NH1]([#6](=O)))]
xN,[N;$([NH0]([*^3])([*^3])([*^3]))&!$([NH0]([#16](=O)))&!$([NH0]([#6](=O)))]
yNH2,[N;$([NH2][#6^2])&!$([NH2]([#6](=O)))&!$([NH2]([#16](=O)))]
yNH,[N;$([NH1]([#6^2])([*^3]))&!$([NH1]([#6](=O)))&!$([NH1]([#16](=O)))]
yN,[N;$([NH0]([#6^2])([*^3])([*^3]))&!$([NH0]([#6](=O)))&!$([NH0]([#16](=O)))]
yyNH,[N;$([NH1]([#6^2])([#6^2]))&!$([NH1]([#6](=O)))]
yyN,[N;$([NH0]([#6^2])([#6^2])([#6^3]))]
yyyN,[N;$([NH0]([#6^2])([#6^2])([#6^2]))]
xC=ONH,[C;$([CH0](=O)([#6^3])([NH1]A))]
xC=ONH2,[C;$([CH0](=O)([#6^3])([NH2]))]
yC=ONH,[C;$([CH0](=O)([#6^2])([NH1]))]
yC=ONH2,[C;$([CH0](=O)([#6^2])([NH2]))]
yNHC=O,[C;$([CH0](=O)([#6^3])([NH1]a))]
yNHC=ON,[C;$([CH0](=O)([NH0])([NH1]a))]
yNHC=ONH,[C;$([CH0](=O)([NH1]A)([NH1]a))&!$([CH0](=O)([NH1]([#16]))([NH1]a))]
yNHC=ONH2,[C;$([CH0](=O)([NH2])([NH1]a))]
ySO2NH2,[S;$([SH0](=O)(=O)([NH2])([#6^2]))]
ySO2NH,[S;$([SH0](=O)(=O)([NH1]A)([#6^2]))&!$([SH0](=O)(=O)([NH1]([CH0](=O)([NH1]([#6^2]))))([#6^2]))]
ySO2N,[S;$([SH0](=O)(=O)([NH0]([#6^3])([#6^3]))([#6^2]))]
xCN,[C;$(C(#N)[#6^3])]
yCN,[C;$(C(#N)[#6^2])]
bip,[c;$([cr6]-[cr6])]
yOC=O,[C;$([CH0](=O)([#6^3])([OH0]([#6^2])))&!$([CR1H0](=O)([#6^3])(O([#6^2])))&!$([CH0](=O)([#6^3])(O([CH0](=O)([#6^3]))))]
xOC=O,[C;$([CH0](=O)([#6^3])([OH0]A))&!$([CR1H0](=O)([#6^3])(O([#6^2])))&!$([OH0]([CH0](=O)([#6^3]))([CH0](=O)([#6^3])))&!$([CR1H0](=O)([#6^3])(O([#6^3])))&!$([CH0](=O)([#6^3])(O([CH0](=O)([#6^3]))))]
xOC=OO,[C;$([CH0](=O)([OH0]A)([OH0]A))&!$([CR1H0](=O)(O)(O))]
yOC=OO,[C;$([CH0](=O)([OH0]*)([OH0]a))]
xCHO,[C;$([CH](=O)[#6^3])]
yCHO,[C;$([CH](=O)([#6^2]))]
yNHC=OO,[C;$([CH0](=O)([OH0]A)([NH1]a))]
yyC=,[C;$([CH0]([#6^2])([#6^2])=[#6])&!$([CR1H0](*)(*)=A)]
yyC=OO,[C;$([CH0](=O)([#6^2])([OH0]([#6^2])))&!$([CH0](=O)([OH0]*)([OH0]a))&!$([CR1H0](=O)([#6^2])(O([CH0](=O)([#6^2]))))]
xC=OOC=O,[O;$([OH0]([CH0](=O)([#6^3]))([CH0](=O)([#6^3])))&!$([OR1H0]([CH0](=O)([#6^2]))([CH0](=O)([#6^2])))]
ryyC=OOC=O,[O;$([OR1H0]([CH0](=O)([#6^2]))([CH0](=O)([#6^2])))]
ryC=OO,[O;$([OR1H0]([CH0](=O)([#6^3]))([#6^2]))&!$([OR1H0]([CH0](=O)([#6^2]))([CH0](=O)([#6^2])))]
rC=OO,[O;$([OR1H0]([CH0](=O)([#6^3]))([#6^3]))]
xC=ON,[C;$([CH1](=O)([NH0]([#6^3])([#6^3])))]
xOCHO,[O;$([OH0]([#6^3])([CH1](=O)))]
ayC,[c;$([cH0]([#6^2])(a)(a))&!$([cR2H0])]
azC,[c;$([cH0]([#6^1])(a)(a))&!$([cR2H0])]
ryC,"[C;$([CR1H0](A)(A)([CH0,CH1](=C))([#6^3])),$([CR1H0]([#6^2])(A)(A)(A))&!$([CR1H0](a))]"
ryCH,"[C;$([CR1H1](A)([CH0,CH1](=C))([#6^3]))]"
ryCH2,"[C;$([CR1H2]([CH0,CH1](=C))([#6^3]))]"
ryyC,"[C;$([CR1H0](A)(A)([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryyCH,"[C;$([CR1H1](A)([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryyCH2,"[C;$([CR1H2]([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryyyC,"[C;$([CR1H0](A)([CH0,CH1](=C))([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryyyCH,"[C;$([CR1H1]([CH0,CH1](=C))([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryyyyC,"[C;$([CR1H0]([CH0,CH1](=C))([CH0,CH1](=C))([CH0,CH1](=C))([CH0,CH1](=C)))]"
rryC,"[C;$([CR2H0]([CH0,CH1](=C))([#6^3])([#6^3])([#6^3]))]"
rryCH,"[C;$([CR2H1]([CH0,CH1](=C))([#6^3])([#6^3]))]"
rryyC,"[C;$([CR2H0]([#6^3])([#6^3])([CH0,CH1](=C))([CH0,CH1](=C)))]"
rryyyC,"[C;$([CR2H0]([#6^3])([CH0,CH1](=C))([CH0,CH1](=C))([CH0,CH1](=C)))]"
ryCH=,[C;$([CR1H1]([#6^2])=A)]
ryC=,[C;$([CR1H0](A)([#6^2])=A)&!$([CR1H0](*)(*)=O)]
rzC,[C;$([CR1H0](A)([#6^3])([#6^3])([#6^1]))]
rzCH,[C;$([CR1H1]([#6^3])([#6^3])([#6^1]))]
rrzC,[C;$([CR2H0](A)([#6^3])([#6^3])([#6^1]))]
ryzC,"[C;$([CR1H0](A)(A)([CH0,CH1](=C))([#6^1]))]"
ryzCH,"[C;$([CR1H1](A)([CH0,CH1](=C))([#6^1]))]"
arC,"[c;$([cR2H0](C([#6^3]))(c([#6^3]))(a)),$([cR2H0]([CH0,CH1](=C([CH0,CH1])))(a)([#6^3])),$([cR2H0](a(a))(a(a))(A))]"
aryC,"[c;$([cR2H0](a)(c([#6^3]))([CH0,CH1](=C([CH0,CH1]))))]"
aarC,"[c;$([cR3H0](c([#6^3]))(c([#6^3]))(a)),$([cR3H0](c([#6^3]))([CH0,CH1](=C([CH0,CH1])))(a))]"
arrC,"[c;$([cR3H0](C([#6^3]))(c([#6^3]))([CH0,CH1](=C([CH0,CH1])))),$([cR3H0](a([#6^3])(a([#6^3])([#6^3])))),$([cR3H0]([#6^3])([CH0,CH1](=C([CH0,CH1])))([CH0,CH1](=C([CH0,CH1]))))]"
rrrC,[C;$([CR3H0](A))]
rrrCH,[C;$([CR3H1])]
rrrrC,[C;$([CR4H0])]
rOC=OO,[C;$([CR1H0](=O)(O)(O))&!$([CR1H0](=O)(O)([#6^2])([CH0](=O)([#6^2])))]
ryyC=O,[C;$([CR1H0](=O)([#6^2])([#6^2]))]
ryC=O,[C;$([CR1H0](=O)([#6^2])([#6^3]))]
rC=O,[C;$([CR1H0](=O)([#6^3])([#6^3]))]
xNHNO2,[N;$([NH1]([NH0](=O)[O-])([#6^3]))]
xNNO2,[N;$([NH0]([NH0](=O)[O-])([#6^3])([#6^3]))]
yNHNO2,[N;$([NH1]([NH0](=O)[O-])([#6^2]))]
yNNO2,[N;$([NH0]([NH0](=O)[O-])([#6^2])([#6^3]))]
rNNO2,[N;$([NR1H0]([NH0](=O)[O-])([#6^3])([#6^3]))]
xCH=NOH,[C;$([CH1]([#6^3])(=N([OH1])))]
xC=NOH,[C;$([CH1]([#6^3])([#6^3])(=N([OH1])))]
yCH=NOH,[C;$([CH1]([#6^2])(=N([OH1])))]
yC=NOH,[C;$([CH1]([#6^2])([#6^3])(=N([OH1])))]
yyCH=N,[C;$([CH1]([#6^2])(=N([#6^2])))]
rCH=NOH,[C;$([CR1H0](=N([OH1])))]
yOC=ONH,[C;$([CH0](=O)([OH0]([#6^2]))([NH1]([#6^3])))]
xOC=ONH,[C;$([CH0](=O)([OH0]([#6^3]))([NH1]([#6^3])))]
xNC=ONH2,[C;$([CH0](=O)([NH0]([#6^3])([#6^3]))([NH2]))]
xNHC=ONH2,[C;$([CH0](=O)([NH1]([#6^3]))([NH2]))]
xNHC=ONH,[C;$([CH0](=O)([NH1]([#6^3]))([NH1]([#6^3])))]
xNC=ON,[C;$([CH0](=O)([NH0]([#6^3])([#6^3]))([NH0]([#6^3])([#6^3])))]
yySO2NHCONH,[S;$([SH0](=O)(=O)([NH1]([CH0](=O)([NH1]([#6^2]))))([#6^2]))]
zC=,[C;$([CH0]([#6^1])([A])=A)&!$([CR1H0]=*)&!$([CH0]([#6^1])([A^2])=A)&!$([CH0]([#6^1])([A^3])=A)]
yzC=,[C;$([CH0]([#6^1])([A^2])=A)]
zC#,[C;$([CH0](#*)([#6^1]))]

"""
rules_csv = StringIO(rules)
rules_df = pd.read_csv(rules_csv,names=["Name","SMARTS"])
rules_df['pat'] = rules_df.SMARTS.apply(Chem.MolFromSmarts)
rules_df

,Name,SMARTS,pat
0,xCH3,[C;$([CH3](A))],<rdkit.Chem.rdchem.Mol object at 0x7f5ee9c89890>
1,xCH2,[C;$([CH2](A))&!$([CH2]([#6^2])(A))&!$([CH2]([...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9c89900>
2,xCH,[C;$([CH1](A))&!$([CH1]([#6^2])(A)(A))&!$([CH1...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9c890b0>
3,xC,[C;$([CH0](A))&!$([CR1H0]*)&!$([CH0]([#6^1]))&...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9c89040>
4,xF,[F;$([F](A))&!$([F]([#6^2]))&!$([F]([#6^1]))],<rdkit.Chem.rdchem.Mol object at 0x7f5ee9c89970>
...,...,...,...
164,xNC=ON,[C;$([CH0](=O)([NH0]([#6^3])([#6^3]))([NH0]([#...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9bba3c0>
165,yySO2NHCONH,[S;$([SH0](=O)(=O)([NH1]([CH0](=O)([NH1]([#6^2...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9bba430>
166,zC=,[C;$([CH0]([#6^1])([A])=A)&!$([CR1H0]=*)&!$([C...,<rdkit.Chem.rdchem.Mol object at 0x7f5ee9bba4a0>
167,yzC=,[C;$([CH0]([#6^1])([A^2])=A)],<rdkit.Chem.rdchem.Mol object at 0x7f5ee9bba510>


Define a standard set of atom types

In [ ]:
name_list = rules_df.Name
name_dict = dict(zip(name_list,list(name_list.index)))

Define a function to report atom types

In [ ]:
def assign_atom_types(mol, name_dict):
    match_list = [""] * mol.GetNumAtoms()
    for name, smarts, pat in rules_df.values:
        matches = [x[0] for x in mol.GetSubstructMatches(pat)]
        for m in matches:
            match_list[m] = name
    clear_substructure_match(mol)
    count_dict = Counter(match_list)
    count_list = [0] * len(name_dict)
    for k,v in count_dict.items():
        if len(k):
            idx = name_dict[k]
            count_list[idx] = v
    return count_list

Make a test dataframe with the first 100 rows 

In [ ]:
test_df = df.head(5000)

Batch process molecules from the Excel file

In [ ]:
type_list = []
for smiles in test_df.smile.values:
    mol = Chem.MolFromSmiles(smiles)
    type_list.append(assign_atom_types(mol,name_dict))

[20:45:46] Conflicting single bond directions around double bond at index 3.
[20:45:46]   BondStereo set to STEREONONE and single bond directions set to NONE.


Put all the data togethter

In [ ]:
res_df = pd.DataFrame(type_list,columns=name_list)
name_df = pd.DataFrame()
name_df['SMILES'] = test_df.smile
name_df['Name'] = test_df.Name
res_df = pd.concat([name_df,res_df],axis=1)
res_df

,SMILES,Name,xCH3,xCH2,xCH,xC,xF,yF,zF,xCl,...,yOC=ONH,xOC=ONH,xNC=ONH2,xNHC=ONH2,xNHC=ONH,xNC=ON,yySO2NHCONH,zC=,yzC=,zC#
0,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC,n-triacontane,2,28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CCCCOC(=O)c1ccccc1C(=O)OCCCC,Dibutyl phthalate,2,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CCCCCCCCCCCCCCCCCCCCCCCCCCCCC,n-nonacosane,2,27,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CCCCCCCCCCCCCCCCCCCCCCCCCCCC,n-octacosane,2,26,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CCCCCCCCCCCCCCCCCCCCCCCCCCC,n-heptacosane,2,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,FC(F)(F)Cl,Chlorotrifluoromethane,0,0,0,1,3,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1266,CC,Ethane,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1267,C=C,Ethene,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1268,FC(F)(F)F,Tetrafluoromethane,0,0,0,1,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Write the data to csv

In [ ]:
df


,name_x,smile_x,dhv at tb median,dhm at Tm median
0,propane,CCC,19.038848,3.520
1,n-butane,CCCC,22.440000,6.965
2,isobutane,CC(C)C,21.300000,4.560
3,n-pentane,CCCCC,25.790000,8.400
4,2-methylbutane,CCC(C)C,24.690000,5.130
...,...,...,...,...
533,Tetralin,C1CCc2ccccc2C1,42.351000,12.450
534,"trans-1,3-Dimethylcyclohexane",C[C@H]1CCC[C@H](C)C1,33.390000,9.870
535,"trans-1,3-dimethylcyclopentane",C[C@@H]1CC[C@@H](C)C1,30.799521,7.400
536,Trichloroethylene,ClC=C(Cl)Cl,31.395000,8.450


In [ ]:
res_df.to_csv("results.csv")

In [ ]:
di=pd.read_excel("/content/sympaper-boil database.xlsx")
di['mol']=di.smile.apply(Chem.MolFromSmiles)
di['mw']=di.mol.apply(MolWt)
di.to_excel('molwt.xlsx')


[20:46:59] Conflicting single bond directions around double bond at index 3.
[20:46:59]   BondStereo set to STEREONONE and single bond directions set to NONE.


In [ ]:
dg = pd.read_excel("/content/11_9_22 no secondary desc.xlsx")
dh=  pd.read_excel("/content/doaa original data.xlsx")

In [ ]:
result = pd.merge(dg, dh, on='Name')
result.to_excel('out.xlsx')

In [ ]:
import warnings
import pandas as pd
import numpy as np

def appendCol(self):
        try:
            self.dg
            self.dh
        except AttributeError:
            return

        joinCol = self.selectColName
        if (self.selectColName == "Name"):
            joinCol = self.selectColName + " lower"
            self.dg[joinCol] = self.dg[self.selectColName].str.lower()
            self.dh[joinCol] = self.dh[self.selectColName].str.lower()

        self.product = pd.merge(self.dg, self.dh, on=joinCol, how='left')
        self.product.to_excel('output.xlsx')

In [ ]:
import warnings
import pandas as pd
import numpy as np

class CsvParser:
    selectColName = "Name"
    librarySheetName = "library"
    targetSheetName = "target"
    # given a .xlsx file, parse it into metadata
    def __init__(self, file_path):
        if ("/content/11_8_22 results.xlsx"!= ""):
            self.librarySheet = pd.read_excel("/content/doaa original data.xlsx", sheet_name=self.librarySheetName)
            self.targetSheet = pd.read_excel("/content/11_8_22 results.xlsx", sheet_name=self.targetSheetName)
            
   

In [ ]:
 # produce an excel file
def produce_output(self):
        try:
            self.product
        except AttributeError:
            return

        writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter');
        self.product.to_excel(writer, sheet_name="merged", index=False)
        writer.to_excel('output.xlsx')

def appendCol(self):
        try:
            self.librarySheet
            self.targetSheet
        except AttributeError:
            return

        joinCol = self.selectColName
        if (self.selectColName == "Name"):
            joinCol = self.selectColName + " lower"
            self.targetSheet[joinCol] = self.targetSheet[self.selectColName].str.lower()
            self.librarySheet[joinCol] = self.librarySheet[self.selectColName].str.lower()

        self.product = pd.merge(self.targetSheet, self.librarySheet, on=joinCol, how='left')

        if (self.selectColName == "Name"):
            self.product.drop(joinCol, inplace=True, axis=1)
        self.to_excel('output.xlsx')